 ## Import Libraries 

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

In [ ]:
import warnings
warnings.filterwarnings('ignore')

## Data Ingestion files

### Testutility file

In [ ]:
%%writefile testutility.py
import logging
import os
import subprocess
import yaml 
import pandas as pd
import datetime
import gc
import re

#################
# File Reading #
#################

def read_config_file(filepath):
  with open(filepath, "r") as stream:
    try:
      return yaml.safe_load(stream)
    except yaml.YAMLError as exc:
      logging.error(exc)

### Yaml file

In [ ]:
%%writefile file.yaml
file_type: csv
dataset_name: testfile
file_name: bank-additional-full
table_name: edsurv
inbound_delimeter: ";"
outbound_delimeter: "|"

## Prepare Data

In [ ]:
# Read Config file
import testutility as util
config_data = util.read_config_file("file.yaml")

In [ ]:
config_data["file_name"]

In [ ]:
# read the files using config file
file_type = config_data["file_type"]
source_file = "./" + config_data["file_name"] + f".{file_type}"

# print("", source_file)
df = pd.read_csv(source_file, config_data["inbound_delimeter"])

In [ ]:
df.shape

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
import os
def summary(df, source_file):
    rows = len(df)
    columns = len(df.columns)
    print(f"Number of Rows: {rows}")
    print(f"Number of Columns: {columns}")
    file_size = os.path.getsize(source_file)
    print(f"Size: {file_size} bytes")

In [ ]:
summary(df, source_file)

In [ ]:
df.columns

In [ ]:
df.duplicated().sum()

Here we have 41k records. So removing record will not affect our existing dataset. So we have drop 12 duplicate records.

In [ ]:
df = df.drop_duplicates()

In [ ]:
df.dtypes

In [ ]:
df.duplicated().sum()

In [ ]:
df.isna().sum()

Now, as per above result,  we can see there is no any duplicate value nor null value.

## Understand Data

### Variable Types

#### Target Variable

In [ ]:
target_var = ['y']
target = df[target_var]
target_var

In [ ]:
target.value_counts(normalize=True).plot(kind="bar", ylabel = "% outcomes", title = "Target y")

The data suggest an imbalance in the target variable. This can affect the performance of our model so an oversampling technique must be applied before model building. 

#### Categorical Variables

In [ ]:
categorical_var = [var for var in df.columns 
                   if df[var].dtype=="O" and 
                   var not in target_var and 
                   var not in ["month"]]
categorical_var

In [ ]:
# let's explore the values of these categorical variables
for var in categorical_var:
    print(var, df[var].unique())
    print()

In [ ]:
for var in categorical_var:
    
    plt.figure(figsize=(12,4))
    
    # plot transformed variable vs sale price
    df[var].value_counts(normalize = True).plot(
    kind="bar",
    xlabel=f"{var}",
    ylabel="% Phone calls",
    title=f"Calls by {var}"
    )
                
    plt.show()

#### Numeric Variables

In [ ]:
numerical_var = [var for var in df.columns if var not in categorical_var + target_var]
numerical_var

#### Temporal Variables 

In [ ]:
temporal_var = [var for var in df.columns if var =="month" or var=="day_of_week"]
temporal_var

#### Discreete Variable 

In [ ]:
discrete_var = [var for var in numerical_var if len(df[var].unique()) < 32 and var not in temporal_var]
discrete_var

##### Continuous variables 

In [ ]:
# make list of continuous variables
continuous_var = [
    var for var in numerical_var if var not in discrete_var+temporal_var]
continuous_var

In [ ]:
# let's explore the values of these temporal variables

for var in continuous_var:
    print(var, df[var].unique())
    print()

In [ ]:
# lets plot histograms for all continuous variables
df[continuous_var].hist(bins=30, figsize=(15,15))
plt.show()

In [ ]:
# let's explore the values of these discrete variables

for var in discrete_var:
    print(var, df[var].unique())
    print()

In [ ]:
for var in discrete_var:
    
    plt.figure(figsize=(12,4))
    
    # plot transformed variable vs sale price
    df[var].value_counts(normalize = True, sort = False).plot(
    kind="bar",
    xlabel=f"{var}",
    ylabel="% Phone Calls",
    title=f"Calls by {var}"
    )
                
    plt.show()

In [ ]:
# let's explore the values of these temporal variables

for var in temporal_var:
    print(var, df[var].unique())
    print()

In [ ]:
for var in temporal_var:
    
    plt.figure(figsize=(12,4))
    
    # plot transformed variable vs sale price
    df[var].value_counts(normalize = True, sort = False).plot(
    kind="bar",
    xlabel=f"{var}",
    ylabel="% Phone Calls",
    title=f"Calls by {var}"
    )
                
    plt.show()

## Cleaning and Transformation

NOTES:
- Duration is a variable we can explore during EDA for business purposes to decide if we should try engaging the person for longer time on the call or not. This variable should not be included in the model becuase Duration is obtained after the call is made to the potential client so if the target client has never received calls, this feature is not very useful.
- month mapped to numerical values and another column of day-month can be generated for EDA in furture.
- No missing values but there are some with unknown values in loan housing etc. Consider deleting them
- There is a 999 value existing in pdays column meaning the customer has not been contacted before. We should change it to 0
- Education column has some values which contain "." character which can be cleaned.
- Outliers need to be handled
- Age column can be log transformed

### Cleaning Education values

In [ ]:
df['education'].replace({'basic.4y' : 'basic 4y', 'basic.6y' : 'basic 6y', 'basic.9y' : 'basic 9y', 'high.school' : 'high school', 'professional.course' : 'professional course', 'university.degree' : 'university degree'}, inplace = True)
df['job'].replace({'admin.' : 'admin'}, inplace = True)
df.head(n=40)

In [ ]:
df['education'].unique()

### Missing Values

In [ ]:
df.isnull().sum()

Fortunately, there are no missing values in the dataset. However, after exploring we find that 5 of the categorical variables have an "unknown" value. Those are the only missing values which do not need to be dealt with for now as the "unknown" category is already created for them. However, deleting them will not affect the dataset so much as they constitute a very small proportion of the dataset. So, we will consider deleting them.

In [ ]:
# To handle the missing values, we first turn the 'unknown' observations to NaNs
df = df.replace('unknown', np.nan)
df.isnull().sum()

In [ ]:
#using partial deletion of the missing values on the job and marital attributes as those attributes do not show a strong relationship between the known/unknown status and target response.
df = df.dropna(subset=['job', 'marital'])

### Mapping month column

In [ ]:
m = {'jan':1, 'feb':2, 'mar':3, 'apr':4, 'may':5, 'jun':6, 'jul':7, 'aug':8, 'sep':9, 'oct':10, 'nov':11, 'dec':12}

In [ ]:
df["month"] = df["month"].map(m)

In [ ]:
df.head()

### Creating month-day column

In [ ]:
df["month-day"] = df['month'].astype('str') + "-" + df['day_of_week'].astype('str')
df.head()

### pdays column value correction

In [ ]:
df['pdays'].mask(df['pdays'] == 999, 0, inplace=True)

In [ ]:
df.head()

# Outlier

we are using box plot method to detect outlier.
Outlier points</b> = Q3 + 1.5 IQR(Upper Quartile) , Q1 - 1.5 IQR(Lower Quartile)

In [ ]:
numerical_var

In [ ]:
fig,axes = plt.subplots(4, 3, figsize=(18, 20))
fig.suptitle('Boxplot graph')
count = 0;
for i in range(4):
    for j in range(3):
        if( count < len(numerical_var)):
            sns.boxplot(ax=axes[i,j], x=numerical_var[count], data=df)
            count = count + 1

From the above Boxplot graph, we can clearly see that age, duration, campaign and cons.conf.idx contain outlier. So we can perform opratoin to remove outlier.

#### Age column log-transformed

In [ ]:
# There are several outliers in the age column. This can be handled by transforming them
# Transorming the outliers using log so the distribution can become more normally distributed
df['log_age'] = [np.log(x) for x in df['age']]

In [ ]:
fig = plt.figure(figsize =(10, 7))
 
# Creating plot
plt.boxplot(df["log_age"])
 
# show plot
plt.show()

#### Removing Outliers 

In [ ]:
# here we have defined generic function to remove outlier from any column with any dataset
def removeOutlier(dataset:pd.DataFrame, column:str, remove: bool, lowerCutoff, upperCutoff):
    Q1 = dataset[column].quantile(lowerCutoff / 100)
    Q3 = dataset[column].quantile(upperCutoff / 100)
    
    IQR = Q3-Q1
    lower_limit = Q1 - IQR*1.5
    upper_limit = Q3 + IQR*1.5
    
    # here, we have tow solution. Ether we can remove the outlier or replace it with upper limit.
    if remove:
        temp = dataset[(dataset[column] < lower_limit) or (dataset[column] > upper_limit)]
    else:
        dataset.loc[dataset[column] < lower_limit, column] = lower_limit 
        dataset.loc[dataset[column] > upper_limit, column] = upper_limit
#         We can also use Replace values of Given column by using np.where() function.
#         dataset[column] = np.where(dataset[column] > upper_limit, upper_limit, dataset[column])

In [ ]:
df[df['duration'] > 3500]['duration'].min()

In [ ]:
# here lower side there is no outlier but upper side, it is clearly visible that after 3500 the frequency descresed. 
# we replace 3500+ value with 3500 value.
# duration(df, 'duration', False, 0, 95)
upper_limit = 3500
df.loc[df['duration'] > upper_limit, 'duration'] = upper_limit 

In [ ]:
df[df['campaign'] > 34]['campaign']

In [ ]:
# here lower side there is no campaign but upper side, it is clearly visible the gap in value after 35. So we set 35 as cutoff value 
# we replace 3500+ value with 3500 value.
# duration(df, 'duration', False, 0, 95)
upper_limit = 34
df.loc[df['campaign'] > upper_limit, 'campaign'] = upper_limit 

In [ ]:
# here lower side there is no outlier but upper side, it is slightly noticable so we are taking 95% data.
upper_limit = -30
df.loc[df['cons.conf.idx'] > upper_limit, 'cons.conf.idx'] = upper_limit 

In [ ]:
fig,axes = plt.subplots(4, 3, figsize=(18, 20))
fig.suptitle('Boxplot graph')
count = 0;
for i in range(4):
    for j in range(3):
        if( count < len(numerical_var)):
            sns.boxplot(ax=axes[i,j], x=numerical_var[count], data=df)
            count = count + 1

In above graph, we can see that we have hanlded outlier point at some extent.